In [1]:
import urllib.request

url = "https://zenodo.org/record/5226945/files/cats_dogs_light.zip?download=1" # Replace with your dataset URL
filename = "cats_dogs_light.zip" # Replace with your desired filename

urllib.request.urlretrieve(url, filename)

('cats_dogs_light.zip', <http.client.HTTPMessage at 0x7dbefe411630>)

In [2]:
import zipfile

with zipfile.ZipFile("/content/cats_dogs_light.zip", 'r') as zip_ref:
    zip_ref.extractall()


this below code makes a train folder containing dogs and cats folders containg training images

In [5]:
import os
import shutil

# Path to the folder containing dog and cat images
source_folder = '/content/cats_dogs_light/train'

# Create a new folder for organizing images
destination_folder = '/content/train'
os.makedirs(destination_folder, exist_ok=True)

# Create subfolders for dogs and cats
dogs_folder = os.path.join(destination_folder, 'dogs')
cats_folder = os.path.join(destination_folder, 'cats')
os.makedirs(dogs_folder, exist_ok=True)
os.makedirs(cats_folder, exist_ok=True)

# Iterate through images in the source folder
for filename in os.listdir(source_folder):
    # Check if the file is an image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Determine if it's a dog or cat image
        if filename.startswith('dog'):
            # Move dog image to the dogs folder
            shutil.move(os.path.join(source_folder, filename), os.path.join(dogs_folder, filename))
        elif filename.startswith('cat'):
            # Move cat image to the cats folder
            shutil.move(os.path.join(source_folder, filename), os.path.join(cats_folder, filename))

print("Organizing images completed.")


Organizing images completed.


this below code makes a test folder containing dogs and cats folders containg test images

In [6]:
import os
import shutil

# Path to the folder containing dog and cat images
source_folder = '/content/cats_dogs_light/test'

# Create a new folder for organizing images
destination_folder = '/content/test'
os.makedirs(destination_folder, exist_ok=True)

# Create subfolders for dogs and cats
dogs_folder = os.path.join(destination_folder, 'dogs')
cats_folder = os.path.join(destination_folder, 'cats')
os.makedirs(dogs_folder, exist_ok=True)
os.makedirs(cats_folder, exist_ok=True)

# Iterate through images in the source folder
for filename in os.listdir(source_folder):
    # Check if the file is an image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Determine if it's a dog or cat image
        if filename.startswith('dog'):
            # Move dog image to the dogs folder
            shutil.move(os.path.join(source_folder, filename), os.path.join(dogs_folder, filename))
        elif filename.startswith('cat'):
            # Move cat image to the cats folder
            shutil.move(os.path.join(source_folder, filename), os.path.join(cats_folder, filename))

print("Organizing images completed.")


Organizing images completed.


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define CNN architecture
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Load and preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        '/content/train',
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
        '/content/test',
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        '/content/test',
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=50)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator, steps=50)
print('Test accuracy:', test_acc)


Found 1000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Epoch 1/20
 50/100 [==============>...............] - ETA: 43s - loss: 0.7027 - accuracy: 0.5300

20/50 [===========>..................] - ETA: 8s - loss: 0.6931 - accuracy: 0.5000

50/50 [==============================] - 6s 109ms/step - loss: 0.6931 - accuracy: 0.5000
Test accuracy: 0.5


In [9]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(classifier, img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.
    prediction = classifier.predict(img_array)
    return "dog" if prediction[0][0] < 0.5 else "cat"

In [12]:
predict_image(model,"/content/test/cats/cat.9975.jpg")

1/1 [==============================] - 0s 31ms/step


'cat'

In [13]:
predict_image(model,"/content/test/dogs/dog.9784.jpg")

1/1 [==============================] - 0s 60ms/step


'cat'